# Подготовка данных для обучения

In [36]:
import pandas as pd

data_path = 'data/credit_history_all.csv'

df = pd.read_csv(data_path)

In [37]:
del df['created_timestamp']
del df['event_timestamp']
del df['loan_id']
del df['zipcode']
del df['dob_ssn']

In [38]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_status,city,state,...,total_wages,credit_card_due,mortgage_due,student_loan_due,vehicle_loan_due,hard_pulls,missed_payments_2y,missed_payments_1y,missed_payments_6m,bankruptcies
0,52,69996,OWN,3.0,PERSONAL,12000,11.11,0,CUMMINGS,KS,...,9617849,7649,779433,28579,3583,0,0,0,0,0
1,61,10636,RENT,3.0,PERSONAL,4000,12.21,1,CRESSON,TX,...,38291019,5101,1877182,32008,20745,6,1,3,0,0
2,36,69000,RENT,0.0,MEDICAL,15600,10.99,0,PLAINVILLE,GA,...,23478174,4223,1142978,24020,11711,3,5,1,1,2
3,36,48686,RENT,21.0,MEDICAL,12000,13.99,0,CLINTON,NC,...,35505191,6993,592528,40529,17508,1,1,0,0,0
4,36,44000,RENT,4.0,DEBTCONSOLIDATION,8000,17.49,1,NASHVILLE,TN,...,571965786,1552,1786677,26664,19749,8,2,0,1,0


In [39]:
# df.to_csv('data/test_data.csv')

Для проверки в пайплайне с энкодером

In [40]:
X = df[df.columns.drop("loan_status")].copy()
y = df[["loan_status"]].copy()

In [42]:
X.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_amnt', 'loan_int_rate',
       'city', 'state', 'location_type', 'tax_returns_filed', 'population',
       'total_wages', 'credit_card_due', 'mortgage_due', 'student_loan_due',
       'vehicle_loan_due', 'hard_pulls', 'missed_payments_2y',
       'missed_payments_1y', 'missed_payments_6m', 'bankruptcies'],
      dtype='object')

In [41]:
X[:5].values

array([[52, 69996, 'OWN', 3.0, 'PERSONAL', 12000, 11.11, 'CUMMINGS',
        'KS', 'PRIMARY', 253, 472, 9617849, 7649, 779433, 28579, 3583, 0,
        0, 0, 0, 0],
       [61, 10636, 'RENT', 3.0, 'PERSONAL', 4000, 12.21, 'CRESSON', 'TX',
        'PRIMARY', 703, 1342, 38291019, 5101, 1877182, 32008, 20745, 6,
        1, 3, 0, 0],
       [36, 69000, 'RENT', 0.0, 'MEDICAL', 15600, 10.99, 'PLAINVILLE',
        'GA', 'PRIMARY', 824, 1578, 23478174, 4223, 1142978, 24020,
        11711, 3, 5, 1, 1, 2],
       [36, 48686, 'RENT', 21.0, 'MEDICAL', 12000, 13.99, 'CLINTON',
        'NC', 'PRIMARY', 1353, 2505, 35505191, 6993, 592528, 40529,
        17508, 1, 1, 0, 0, 0],
       [36, 44000, 'RENT', 4.0, 'DEBTCONSOLIDATION', 8000, 17.49,
        'NASHVILLE', 'TN', 'PRIMARY', 15882, 24868, 571965786, 1552,
        1786677, 26664, 19749, 8, 2, 0, 1, 0]], dtype=object)

In [6]:
categorical_features = [
    "person_home_ownership",
    "loan_intent",
    "city",
    "state",
    "location_type",
]

In [7]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

encoder.fit(df[categorical_features])

OrdinalEncoder()

In [24]:
import joblib

encoder_filename = "encoder.pkl"
_ = joblib.dump(encoder, encoder_filename)

In [8]:
target = "loan_status"

transform_training_df=df.copy()
transform_training_df[categorical_features] = encoder.transform(
    df[categorical_features]
)

train_X = transform_training_df[
    transform_training_df.columns.drop(target)
]
train_X = train_X.reindex(sorted(train_X.columns), axis=1)
train_Y = transform_training_df.loc[:, target]

In [9]:
train_X.head()

,bankruptcies,city,credit_card_due,hard_pulls,loan_amnt,loan_int_rate,loan_intent,location_type,missed_payments_1y,missed_payments_2y,...,person_age,person_emp_length,person_home_ownership,person_income,population,state,student_loan_due,tax_returns_filed,total_wages,vehicle_loan_due
0,0,2062.0,7649,0,12000,11.11,4.0,0.0,0,0,...,52,3.0,2.0,69996,472,16.0,28579,253,9617849,3583
1,0,1998.0,5101,6,4000,12.21,4.0,0.0,3,1,...,61,3.0,3.0,10636,1342,43.0,32008,703,38291019,20745
2,2,6984.0,4223,3,15600,10.99,3.0,0.0,1,5,...,36,0.0,3.0,69000,1578,10.0,24020,824,23478174,11711
3,0,1697.0,6993,1,12000,13.99,3.0,0.0,0,1,...,36,21.0,3.0,48686,2505,27.0,40529,1353,35505191,17508
4,0,6014.0,1552,8,8000,17.49,0.0,0.0,0,2,...,36,4.0,3.0,44000,24868,42.0,26664,15882,571965786,19749


In [10]:
train_Y.head()

0    0
1    1
2    0
3    0
4    1
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.10)

# DecisionTreeClassifier

In [29]:
from sklearn.tree import DecisionTreeClassifier

max_depth = 22

classifier = DecisionTreeClassifier(max_depth = max_depth)
classifier.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=22)

In [30]:
DTC_filename = "DTC.pkl"
_ = joblib.dump(classifier, DTC_filename)

In [13]:
X[categorical_features] = encoder.transform(X[categorical_features])
X = X.reindex(sorted(X.columns), axis=1)

In [14]:
predictions = classifier.predict(X)

In [15]:
from sklearn.metrics import accuracy_score, make_scorer

accuracy = accuracy_score(y_true=y, y_pred = predictions)
print(accuracy)

0.9847056358684265


In [16]:
predictions = classifier.predict(x_test)

accuracy = accuracy_score(y_true=y_test, y_pred = predictions)
print(accuracy)

0.8851256983240223


# GradientBoostingClassifier

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier().fit(x_train, y_train)

In [28]:
GBC_filename = "GBC.pkl"
_ = joblib.dump(clf, GBC_filename)

In [21]:
predictions = clf.predict(X)

In [22]:
from sklearn.metrics import accuracy_score, make_scorer

accuracy = accuracy_score(y_true=y, y_pred = predictions)
print(accuracy)

0.9192331866750472


In [23]:
predictions = clf.predict(x_test)

accuracy = accuracy_score(y_true=y_test, y_pred = predictions)
print(accuracy)

0.910963687150838


In [31]:
predictions

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [32]:
model = joblib.load("DTC.pkl")

In [34]:
predictions = model.predict(X)

In [35]:
predictions

array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [ ]:
class DTCModel(object):  
    
    def __init__(self):
        
        import pandas as pd
        from sklearn.tree import DecisionTreeClassifier
        from sklearn.preprocessing import OrdinalEncoder
        
        self.categorical_features = [
            "person_home_ownership",
            "loan_intent",
            "city",
            "state",
            "location_type",
        ]
        
        self.encoder = joblib.load("encoder.pkl")
        
        print("Encoder loaded")
        
        self.model = joblib.load("DTC.pkl")
        
        print("Model loaded")

    def predict(self,X,features_names):

        df = pd.Dataframe(X, columns=features_names)
        
        df[self.categorical_features] = self.encoder.transform(df[self.categorical_features])
        df = df.reindex(sorted(df.columns), axis=1)
        
        predictions = self.model.predict(df)
        
        return predictions

    def send_feedback(self,features,feature_names,reward,truth,routing):
        """
        Handle feedback

        Parameters
        ----------
        features : array - the features sent in the original predict request
        feature_names : array of feature names. May be None if not available.
        reward : float - the reward
        truth : array with correct value (optional)
        """
        print("Send feedback called")
        return []

In [ ]:
class GBCModel(object):  
    
    def __init__(self):
        
        import pandas as pd
        from sklearn.ensemble import GradientBoostingClassifier
        from sklearn.preprocessing import OrdinalEncoder

        
        self.categorical_features = [
            "person_home_ownership",
            "loan_intent",
            "city",
            "state",
            "location_type",
        ]
        
        self.encoder = joblib.load("encoder.pkl")
        
        print("Encoder loaded")
        
        self.model = joblib.load("GBC.pkl")
        
        print("Model loaded")

    def predict(self,X,features_names):

        df = pd.Dataframe(X, columns=features_names)
        
        df[self.categorical_features] = self.encoder.transform(df[self.categorical_features])
        df = df.reindex(sorted(df.columns), axis=1)
        
        predictions = self.model.predict(df)
        
        return predictions

    def send_feedback(self,features,feature_names,reward,truth,routing):
        """
        Handle feedback

        Parameters
        ----------
        features : array - the features sent in the original predict request
        feature_names : array of feature names. May be None if not available.
        reward : float - the reward
        truth : array with correct value (optional)
        """
        print("Send feedback called")
        return []